In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet18

In [2]:
# Load the CIFAR-10 dataset
path = '/mnt/datasets'

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

trainset = torchvision.datasets.CIFAR10(root = path, train = True, 
                                             transform = transform, download = False)
trainloader = DataLoader(trainset, batch_size = 100, shuffle = True, num_workers = 2)
 
testset = torchvision.datasets.CIFAR10(root = path, train = False, 
                                             transform = transform, download = False)
testloader = DataLoader(testset, batch_size = 100, shuffle = False, num_workers = 2)


In [ ]:
#Define the ResNet architecture
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

model = resnet18(pretrained=False, num_classes=10).to(device)


# Train the model
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.1, momentum = 0.9, weight_decay = 5e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

num_epochs = 100
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    scheduler.step()

    print(f"Epoch {epoch+1}, Loss: {running_loss / (i + 1)}")

print("Finished Training")

Epoch 1, Loss: 2.0772085835933685
Epoch 2, Loss: 1.5157153403759003
Epoch 3, Loss: 1.3099504961967467
Epoch 4, Loss: 1.1714966517686844
Epoch 5, Loss: 1.0782042129039764
Epoch 6, Loss: 1.0164430745840072
Epoch 7, Loss: 0.9684172339439392
Epoch 8, Loss: 0.9324923511743546
Epoch 9, Loss: 0.8930025655031204
Epoch 10, Loss: 0.8725947428941727
Epoch 11, Loss: 0.8603321393728256
Epoch 12, Loss: 0.8469131374359131
Epoch 13, Loss: 0.8317745009660721
Epoch 14, Loss: 0.8205407214164734
Epoch 15, Loss: 0.8130366908311843
Epoch 16, Loss: 0.8045880342721939
Epoch 37, Loss: 0.3934302118122578
Epoch 38, Loss: 0.38055060973763466
Epoch 39, Loss: 0.37114668959379193
Epoch 40, Loss: 0.36356905502080916
Epoch 41, Loss: 0.3607530941367149
Epoch 43, Loss: 0.3500710208117962
Epoch 50, Loss: 0.3324847225546837
Epoch 51, Loss: 0.32637508153915407
Epoch 52, Loss: 0.3287258842587471
Epoch 53, Loss: 0.32636810207366945
Epoch 54, Loss: 0.3268858152627945
Epoch 55, Loss: 0.3140726448595524
Epoch 56, Loss: 0.318224

In [ ]:
#Evaluate the model
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy of the network on the 10000 test images: {100 * correct / total}%")

In [ ]:
torch.save(model, '/mnt/Codes/Resnet18_c10.pth')

In [ ]:
model = torch.load('/mnt/Codes/Resnet18_c10.pth')